<a href="https://colab.research.google.com/github/nvasin/StockAdvisor/blob/main/experiment_with_SES/baseline_SES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

In [148]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"
import math
import ast

In [149]:
from utils import SimpleExponentialSmoothing, plot_ts_forecast, build_forecast, qualityMAPE

## Detailed example with 'TRNFP'

### Load data

In [150]:
def load_data(ticker = 'TRNFP'):
  ts = pd.read_csv(f'{ticker}_moex_10m_data.csv')

  ts['Datetime'] = pd.to_datetime(ts['Datetime'])

  ts.set_index('Datetime', inplace = True)

  ts = ts['Close'].resample("D").sum()

  ts = ts[ts > 0.0]

  ts = pd.DataFrame(ts)

  return ts

In [151]:
ts = load_data()

### Look at data

In [152]:
ts

,Close
Datetime,
2021-10-01,115365.0
2021-10-04,212142.5
2021-10-05,313241.0
2021-10-06,405293.5
2021-10-07,492914.0
...,...
2024-10-29,523452.5
2024-10-30,524907.5
2024-10-31,515077.5


In [153]:
ts.plot().update_layout(height=350, width=1300).show()

### Make prediction

In [154]:
row_num = ts.shape[0]
boundaries = []

for i in range(0, row_num, round(row_num/4)):
  boundaries.append(ts.index[i])

boundaries

[Timestamp('2021-10-01 00:00:00'),
 Timestamp('2022-08-03 00:00:00'),
 Timestamp('2023-05-04 00:00:00'),
 Timestamp('2024-02-01 00:00:00')]

In [155]:
ALPHA = [0.9, 0.7, 0.4, 0.2, .15, 0.1, 0.05, 0.01, 0.001, 0.00001]
ESparams = [{'alpha':alpha} for alpha in ALPHA]
FRC_ts = build_forecast(h=1, ts=ts.loc[boundaries[2]:boundaries[3]], alg_name = 'SimpleExponentialSmoothing', alg_title = 'ES', params = ESparams)

In [156]:
qlt_ses = pd.DataFrame(index = ts.columns, columns = FRC_ts.keys())

ix = ts.loc[boundaries[2]:boundaries[3]].index
for param_cntr in sorted(qlt_ses.columns):
    frc_ts = FRC_ts[param_cntr]
    qlt_ses[param_cntr],_ = qualityMAPE(ts.loc[ix], frc_ts.loc[ix])

qlt_ses[qlt_ses.columns].mean().sort_values()

,0
ES {'alpha': 0.9},0.011184
ES {'alpha': 0.7},0.012091
ES {'alpha': 0.4},0.015787
ES {'alpha': 0.2},0.023460
ES {'alpha': 0.15},0.027911
ES {'alpha': 0.1},0.035666
ES {'alpha': 0.05},0.051994
ES {'alpha': 0.01},0.100197
ES {'alpha': 0.001},0.176125
ES {'alpha': 1e-05},0.190262


In [157]:
alg_name = qlt_ses[qlt_ses.columns].mean().sort_values().index[0]
# plot_ts_forecast(ts.loc[boundaries[2]:boundaries[3]], FRC_ts[alg_name].loc[boundaries[2]:boundaries[3]]
#                , ts_num=0, alg_title='ES alpha=0.9')

In [158]:
qualityMAPE(ts, FRC_ts[alg_name])[0]

,0
Close,0.011132


In [159]:
# forecast delay = 1
h = 1

# data frame to store forecasted values
frc_ts = pd.DataFrame(columns = ts.columns)

# ES params
Params ={'alpha':0.9}

# generate forecasts for each Item
for cntr in ts.columns:
    frc_ts[cntr] = SimpleExponentialSmoothing(ts[cntr], h, Params)

plot_ts_forecast(ts.loc['2021-10-01':'2024-09-02'], frc_ts.loc['2021-10-01':'2024-12-31'], ts_num=0, alg_title='ES alpha=0.9')

In [160]:
temp = plot_ts_forecast(ts.loc['2021-10-01':'2024-09-02'], frc_ts.loc['2021-10-01':'2024-12-31'], ts_num=0, alg_title='ES alpha=0.9')

## Just prediction with other tickers

### Putting everything inside one function

In [161]:
def predict(ticker = 'TRNFP'):

  print(f"\t\t\t\t\t\t ticker:   {ticker}")

  ts = load_data(ticker)

  row_num = ts.shape[0]
  boundaries = []

  for i in range(0, row_num, round(row_num/4)):
    boundaries.append(ts.index[i])

  ALPHA = [0.9, 0.7, 0.4, 0.2, .15, 0.1, 0.05, 0.01, 0.001, 0.00001]
  ESparams = [{'alpha':alpha} for alpha in ALPHA]
  FRC_ts = build_forecast(h=1, ts=ts.loc[boundaries[2]:boundaries[3]],
                          alg_name = 'SimpleExponentialSmoothing',
                          alg_title = 'ES', params = ESparams)

  qlt_ses = pd.DataFrame(index = ts.columns, columns = FRC_ts.keys())

  ix = ts.loc[boundaries[2]:boundaries[3]].index
  for param_cntr in sorted(qlt_ses.columns):
      frc_ts = FRC_ts[param_cntr]
      qlt_ses[param_cntr],_ = qualityMAPE(ts.loc[ix], frc_ts.loc[ix])

  alg_name = qlt_ses[qlt_ses.columns].mean().sort_values().index[0]

  print(f"MAPE: {qualityMAPE(ts, FRC_ts[alg_name])[0].iloc[0]}")

  h = 1

  frc_ts = pd.DataFrame(columns = ts.columns)

  Params = ast.literal_eval(alg_name[3:])

  for cntr in ts.columns:
      frc_ts[cntr] = SimpleExponentialSmoothing(ts[cntr], h, Params)

  plot_ts_forecast(ts.loc[boundaries[0]:boundaries[3]], frc_ts.loc[boundaries[0]:boundaries[3]], ts_num=0, alg_title=f"ES alpha={Params['alpha']}")

### Predictions

In [162]:
predict('YDEX')

						 ticker:   YDEX
MAPE: 0.012018914782375068


In [163]:
predict('VTBR')

						 ticker:   VTBR
MAPE: 0.010930777316680419


In [164]:
predict('UPRO')

						 ticker:   UPRO
MAPE: 0.012254008140464304


In [165]:
predict('TRNFP')

						 ticker:   TRNFP
MAPE: 0.011132374041439649


In [166]:
predict('TATN')

						 ticker:   TATN
MAPE: 0.009040892729301593


In [167]:
predict('TATNP')

						 ticker:   TATNP
MAPE: 0.008443780623991986


In [168]:
predict('SNGS')

						 ticker:   SNGS
MAPE: 0.010967205121680238


In [169]:
predict('SELG')

						 ticker:   SELG
MAPE: 0.016636555112032186


In [170]:
predict('SBER')

						 ticker:   SBER
MAPE: 0.007078081466779326


In [171]:
predict('RUAL')

						 ticker:   RUAL
MAPE: 0.007762581910105224


In [172]:
predict('RTKM')

						 ticker:   RTKM
MAPE: 0.009159439496662813


In [173]:
predict('POSI')

						 ticker:   POSI
MAPE: 0.011345248489924005


In [174]:
predict('PLZL')

						 ticker:   PLZL
MAPE: 0.008016291159683334


In [175]:
predict('OZON')

						 ticker:   OZON
MAPE: 0.01246831607102695


In [176]:
predict('NVTK')

						 ticker:   NVTK
MAPE: 0.008046216647930843


In [177]:
predict('MTSS')

						 ticker:   MTSS
MAPE: 0.007732666024638028


In [178]:
predict('MTLR')

						 ticker:   MTLR
MAPE: 0.015750388208414966


In [179]:
predict('MTLRP')

						 ticker:   MTLRP
MAPE: 0.01715393344868189


In [180]:
predict('MSNG')

						 ticker:   MSNG
MAPE: 0.01883548163243299


In [181]:
predict('MOEX')

						 ticker:   MOEX
MAPE: 0.010011476364600659


In [182]:
predict('MGNT')

						 ticker:   MGNT
MAPE: 0.012007881775819483


In [183]:
predict('LEAS')

						 ticker:   LEAS
MAPE: 0.021987771551491717


In [184]:
predict('IRAO')

						 ticker:   IRAO
MAPE: 0.007180533890531456


In [185]:
predict('HYDR')

						 ticker:   HYDR
MAPE: 0.010684853408122698


In [186]:
predict('GMKN')

						 ticker:   GMKN
MAPE: 0.00710574036749971


In [187]:
predict('GAZP')

						 ticker:   GAZP
MAPE: 0.006044263146912325


In [188]:
predict('FIVE')

						 ticker:   FIVE
MAPE: 0.010678354875788163


In [189]:
predict('FEES')

						 ticker:   FEES
MAPE: 0.013816224127752158


In [190]:
predict('ENPG')

						 ticker:   ENPG
MAPE: 0.011321733122709595


In [191]:
predict('CHMF')

						 ticker:   CHMF
MAPE: 0.010189117329285863


In [192]:
predict('ASTR')

						 ticker:   ASTR
MAPE: 0.014273530411845045


In [193]:
predict('ALRS')

						 ticker:   ALRS
MAPE: 0.010105222380184952


In [194]:
predict('AGRO')

						 ticker:   AGRO
MAPE: 0.016254676442931767


## Comments on metric

In [195]:
# при одинаковом параметре alpha = 0.9,
# метрика MAPE варьируется от 0.006 до 0.022,
# что в процентах 6-22%,
# что свидетельствует о неплохом прогнозе на данных временных рядах,
# что и можно увидеть на графиках.